In [1]:
import openai
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_core.prompts import ChatPromptTemplate
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os
from xml.dom import minidom
from concurrent.futures import ThreadPoolExecutor
import json
from typing import List, Dict, Union
import re
from datetime import datetime, timedelta


# ✅ API 설정
BASIS_URL = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"
DETAIL_URL = "http://apis.data.go.kr/B551182/MadmDtlInfoService2.7/getDtlInfo2.7"
API_KEY = "jB13be2Bs3Cy4foj2au1Y8dbOzJMrEhELtQg54is5jFE9EbDa1TND7ELLBe3L1YJ0rDDD5uPd87JAbGFcbicqA=="

# ✅ 파일 저장 경로
EXCEL_FILE = "hospitals_basic_data.xlsx"
XML_FILE = "zero_coordinates_hospitals.xml"
OUTPUT_FILE = "hospitals_with_time.json"

# ✅ API 요청 설정
NUM_ROWS = 5000  # 한 번에 가져올 데이터 개수
TOTAL_COUNT = 78484  # 총 병원 개수 (API 문서 기준)
TOTAL_PAGES = (TOTAL_COUNT // NUM_ROWS) + 1  # 필요한 총 페이지 수
MAX_WORKERS = 10  # 병렬 처리 스레드 개수

# ✅ 사용자 위치 (임시로 강남/영등포)
USER_LAT, USER_LON = 37.525782, 126.883021  # 영등포

# ✅ 반경 설정 (3km)
RADIUS_M = 3000

# ✅ 위/경도 0인 병원 저장 리스트
zero_coordinates_hospitals = []

# ✅ 진료과목 코드 딕셔너리
medical_departments = {
    "일반의": "00",
    "내과": "01",
    "신경과": "02",
    "정신건강의학과": "03",
    "외과": "04",
    "정형외과": "05",
    "신경외과": "06",
    "심장혈관흉부외과": "07",
    "성형외과": "08",
    "마취통증의학과": "09",
    "산부인과": "10",
    "소아청소년과": "11",
    "안과": "12",
    "이비인후과": "13",
    "피부과": "14",
    "비뇨의학과": "15",
    "영상의학과": "16",
    "방사선종양학과": "17",
    "병리과": "18",
    "진단검사의학과": "19",
    "결핵과": "20",
    "재활의학과": "21",
    "핵의학과": "22",
    "가정의학과": "23",
    "응급의학과": "24",
    "직업환경의학과": "25",
    "예방의학과": "26",
    "기타1(치과)": "27",
    "기타4(한방)": "28",
    "기타2": "31",
    "기타2(2)": "40",
    "보건": "41",
    "기타3": "42",
    "보건기관치과": "43",
    "보건기관한방": "44",
    "치과": "49",
    "구강악안면외과": "50",
    "치과보철과": "51",
    "치과교정과": "52",
    "소아치과": "53",
    "치주과": "54",
    "치과보존과": "55",
    "구강내과": "56",
    "영상치의학과": "57",
    "구강병리과": "58",
    "예방치과": "59",
    "치과소계": "60",
    "통합치의학과": "61",
    "한방내과": "80",
    "한방부인과": "81",
    "한방소아과": "82",
    "한방안·이비인후·피부과": "83",
    "한방신경정신과": "84",
    "침구과": "85",
    "한방재활의학과": "86",
    "사상체질과": "87",
    "한방응급": "88",
    "한방응급": "89",
    "한방소계": "90",
}

# ✅ 진료시간 정보 컬럼 리스트
COLUMNS = [
    "plcDir",
    "trmtSunEnd",
    "rcvSat",
    "trmtSunStart",
    "emyNgtTelNo1",
    "emyNgtTelNo2",
    "lunchWeek",
    "trmtWedStart",
    "trmtWedEnd",
    "trmtThuStart",
    "trmtMonStart",
    "trmtMonEnd",
    "trmtTueStart",
    "trmtTueEnd",
    "lunchSat",
    "rcvWeek",
    "trmtSatStart",
    "trmtSatEnd",
    "plcNm",
    "parkXpnsYn",
    "parkEtc",
    "noTrmtSun",
    "noTrmtHoli",
    "emyDayYn",
    "emyDayTelNo1",
    "emyDayTelNo2",
    "emyNgtYn",
    "trmtThuEnd",
    "trmtFriStart",
    "trmtFriEnd",
    "plcDist",
    "parkQty",
]

# API 키 정보 로드
load_dotenv()

# LangSmith 추적을 설정합니다.
logging.langsmith("Medical-Agent")

# 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """너는 아이케어 챗봇으로 한국의 병원 및 약국 검색 도우미입니다.
            사용자의 증상을 분석하여 적절한 진료과를 추천해 주세요.
            
            - 단, 아이가 아프더라도 무조건 소아과만 추천하지 말고 증상에 따라 추천해주세요.
            - 반드시 결과는 단답형으로 추천 진료과만 반환하고, 형식은 아래 예시처럼 유지하세요.
            
            예시:
            사용자 입력: "목이 아프고 열이 나요."
            챗봇 응답:
            ```
            이비인후과, 내과
            ```
            
            사용자 입력: "무릎이 너무 아파요."
            챗봇 응답:
            ```
            정형외과
            ```
            
            항상 한국어로 응답하고, 친절하고 간결하게 안내해주세요.
            """,
        ),
        ("human", "{input}"),
    ]
)

# LLM 정의
llm = ChatOpenAI(model="gpt-4o", temperature=0)


def get_medical_department(user_input):
    """사용자의 입력을 받아서 적절한 진료과를 추천해주는 함수"""
    if not user_input.strip():  # 빈 입력 처리
        return ["🤖 챗봇: 증상을 입력해주세요."]

    formatted_prompt = prompt.format(input=user_input)
    response = llm.invoke(formatted_prompt)

    if response and response.content:
        # 응답에서 진료과를 리스트 형태로 추출
        content = response.content.strip().strip("```").strip()
        departments = content.split(", ")
        return departments
    else:
        return ["🤖 챗봇: 죄송합니다. 다시 입력해주세요."]


# 사용자 입력을 받아서 적절한 진료과를 추천해주는 함수 호출
user_input = input("어떤 증상으로 방문하시나요? ")
recommended_departments = get_medical_department(
    user_input
)  # 이 부분에서 agent tool 호출
print(recommended_departments)


def fetch_hospital_page(page_no, dgsbjtCd):
    """병렬로 특정 페이지의 병원 데이터를 가져오는 함수"""
    params = {
        "ServiceKey": API_KEY,
        "numOfRows": str(NUM_ROWS),
        "pageNo": str(page_no),
        "dgsbjtCd": dgsbjtCd,
        "xPos": str(USER_LON),
        "yPos": str(USER_LAT),
        "radius": str(RADIUS_M),
    }
    response = requests.get(BASIS_URL, params=params)

    if response.status_code != 200:
        print(f"🚨 API 호출 실패 (페이지 {page_no}): {response.status_code}")
        return []

    # XML → JSON 변환
    root = ET.fromstring(response.content)
    hospitals = []

    for item in root.findall(".//item"):
        xpos = float(item.findtext("XPos", "0"))
        ypos = float(item.findtext("YPos", "0"))

        hospital_data = {
            "ykiho": item.findtext("ykiho", "정보 없음"),  # 병원 고유 ID
            "yadmNm": item.findtext("yadmNm", "정보 없음"),  # 병원명
            "addr": item.findtext("addr", "정보 없음"),  # 주소
            "XPos": xpos,  # 위도
            "YPos": ypos,  # 경도
            "telno": item.findtext("telno", "정보 없음"),  # 전화번호
            "진료과목": [k for k, v in medical_departments.items() if v == dgsbjtCd][
                0
            ],  # 진료과목 한글로 추가
        }

        if xpos == 0 or ypos == 0:
            zero_coordinates_hospitals.append(item)  # ✅ 위경도 0인 병원 저장
        else:
            hospitals.append(hospital_data)  # ✅ 정상 병원만 저장

    return hospitals


def fetch_all_hospitals(dgsbjtCd):
    """병렬 처리로 API에서 모든 병원 데이터를 가져오는 함수"""
    all_hospitals = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(
            executor.map(
                lambda page_no: fetch_hospital_page(page_no, dgsbjtCd),
                range(1, TOTAL_PAGES + 1),
            )
        )

    # ✅ 결과 병합
    for hospitals in results:
        all_hospitals.extend(hospitals)

    print(f"🚀 전체 병원 데이터 수집 완료! 총 {len(all_hospitals)}개 (위/경도 0 제외)")

    return all_hospitals


# 맵핑 및 API 호출
all_hospitals = []
for department in recommended_departments:
    if department in medical_departments:
        dgsbjtCd = medical_departments[department]
        hospitals = fetch_all_hospitals(dgsbjtCd)
        all_hospitals.extend(hospitals)


# 진료시간 정보 추가
def fetch_hospital_details(ykiho):
    """특정 병원의 진료시간 및 추가 정보를 가져오기"""
    params = {"serviceKey": API_KEY, "ykiho": ykiho}
    response = requests.get(DETAIL_URL, params=params)

    if response.status_code != 200:
        return None  # API 오류 시 해당 병원 제외

    root = ET.fromstring(response.content)
    item = root.find(".//item")
    if item is None:
        return None  # 데이터가 없으면 제외

    # ✅ 데이터 추출
    hospital_info = {"ykiho": ykiho}
    for col in COLUMNS:
        hospital_info[col] = item.findtext(col, "정보 없음")  # 데이터 없으면 기본값

    return hospital_info


def fetch_all_hospital_details(df):
    """병렬 처리로 모든 병원의 진료시간 정보 가져오기"""
    print("✅ 병원별 진료시간 정보 가져오는 중...")

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(fetch_hospital_details, df["ykiho"]))

    # ✅ 유효한 데이터만 필터링
    hospital_details = [r for r in results if r is not None]

    # ✅ 데이터프레임 변환
    df_details = pd.DataFrame(hospital_details)

    # ✅ 기존 데이터와 병합
    df_result = df.merge(df_details, on="ykiho", how="left")

    # ✅ 진료시간 관련 컬럼들
    time_columns = [
        "trmtSunEnd",
        "trmtSunStart",
        "trmtWedStart",
        "trmtWedEnd",
        "trmtThuStart",
        "trmtMonStart",
        "trmtMonEnd",
        "trmtTueStart",
        "trmtTueEnd",
        "trmtSatStart",
        "trmtSatEnd",
        "trmtThuEnd",
        "trmtFriStart",
        "trmtFriEnd",
    ]

    # ✅ 진료시간 정보가 하나라도 있는 병원만 필터링
    df_result = df_result[df_result[time_columns].any(axis=1)]

    print(f"📊 병원 진료시간 정보 추가 완료! 총 {len(df_result)}개 병원")

    return df_result


# ✅ all_hospitals 리스트를 DataFrame으로 변환
df_hospitals = pd.DataFrame(all_hospitals)

# ✅ 병원 진료시간 정보 가져오기
df_hospitals_with_details = fetch_all_hospital_details(df_hospitals)

json_file_path = (
    r"C:\Users\RMARKET\workspace\project\backend\chat\test\hospital_details211.json"
)

df_hospitals_with_details.to_json(json_file_path, orient="records", force_ascii=False)

# 파일 경로 반환
json_file_path

Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 21
Python-dotenv could not parse statement starting at line 22
Python-dotenv could not parse statement starting at line 23
Python-dotenv could not parse statement starting at line 24


LangSmith 추적을 시작합니다.
[프로젝트명]
Medical-Agent
['이비인후과', '내과', '소화기내과']
🚀 전체 병원 데이터 수집 완료! 총 158개 (위/경도 0 제외)
🚀 전체 병원 데이터 수집 완료! 총 297개 (위/경도 0 제외)
✅ 병원별 진료시간 정보 가져오는 중...
📊 병원 진료시간 정보 추가 완료! 총 220개 병원


'C:\\Users\\RMARKET\\workspace\\project\\backend\\chat\\test\\hospital_details211.json'

# 위에 코드는 시간까지 불러옴


In [ ]:
import os
import re
import pandas as pd
from datetime import datetime, timedelta
from holidayskr import is_holiday
from typing import Dict, List, Union

# TODO : 병렬처리

# ✅ 파일 경로 설정
DATA_PATH = r"C:\Users\RMARKET\workspace\project\backend\chat\data\hospitals_시간포함_null값제외.xlsx"
OUTPUT_FILE = r"C:\Users\RMARKET\workspace\project\backend\chat\data\processed_hospital_data2.json"


def normalize_lunch_time(lunch_time: str) -> Union[None, List[str]]:
    """점심시간을 ['HH:MM', 'HH:MM'] 형식으로 변환."""
    if not lunch_time or "정보 없음" in lunch_time or "없음" in lunch_time:
        return None

    lunch_time = re.sub(r"[^0-9시분~:-]", "", lunch_time)
    time_match = re.findall(
        r"(\d{1,2})시?(\d{1,2})?분?\s*[~:-]\s*(\d{1,2})시?(\d{1,2})?분?", lunch_time
    )

    if time_match:
        start_hour, start_minute, end_hour, end_minute = time_match[0]
        start_hour = int(start_hour)
        start_minute = int(start_minute) if start_minute else 0
        end_hour = int(end_hour)
        end_minute = int(end_minute) if end_minute else 0
        return [f"{start_hour:02}:{start_minute:02}", f"{end_hour:02}:{end_minute:02}"]

    return None


def normalize_noTrmtSun(noTrmtSun: str) -> bool:
    """일요일 진료 여부를 반환 (휴진이면 True)"""
    if not noTrmtSun or "정보 없음" in noTrmtSun:
        return False
    return any(term in noTrmtSun for term in ["전부휴진", "휴진", "휴무"])


def get_holiday_dates(year: int = 2025) -> List[str]:
    """특정 연도의 모든 공휴일 날짜를 반환"""
    holidays = []
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    current_date = start_date
    while current_date <= end_date:
        if is_holiday(current_date.strftime("%Y-%m-%d")):
            holidays.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    return sorted(holidays)


def normalize_treatment_time(row: pd.Series) -> Dict[str, Union[List[str], None]]:
    """진료시간을 요일별 ['HH:MM', 'HH:MM'] 형식으로 변환"""
    day_mapping = {
        "월": "Mon",
        "화": "Tue",
        "수": "Wed",
        "목": "Thu",
        "금": "Fri",
        "토": "Sat",
        "일": "Sun",
    }
    treatment_info = {}

    for kor_day, eng_day in day_mapping.items():
        start_time = row.get(f"trmt{eng_day}Start", "정보 없음")
        end_time = row.get(f"trmt{eng_day}End", "정보 없음")
        if start_time != "정보 없음" and end_time != "정보 없음":
            treatment_info[kor_day] = [
                f"{start_time[:2]}:{start_time[2:]}",
                f"{end_time[:2]}:{end_time[2:]}",
            ]
        else:
            treatment_info[kor_day] = None

    return treatment_info


# rcvSat - 토요일 접수시간
def normalize_rcvSat(reception_time):
    """토요일 접수시간을 통일된 형식으로 변환 (HH:MM / HH:MM~HH:MM)"""

    if not reception_time or "정보 없음" in reception_time:
        return {"토": "정보 없음"}

    reception_time = reception_time.strip()
    result = {"토": []}

    # ✅ 연중무휴 처리 (토요일만 적용)
    if "연중무휴" in reception_time or "365일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            result["토"].append(f"{start_hour:02}:{start_minute:02}")
            result["토"].append(f"{end_hour:02}:{end_minute:02}")
        else:
            result["토"] = ["연중무휴"]
        result["연중무휴 여부"] = True
        return result

    # ✅ "HH:MM까지 접수" → 마감 시간만 처리
    closing_pattern = re.search(
        r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*까지\s*접수?", reception_time
    )
    if closing_pattern:
        close_hour, close_minute = closing_pattern.groups()
        close_hour = int(close_hour)
        close_minute = (
            int(close_minute) if close_minute and close_minute.isdigit() else 0
        )
        result["토"].append(f"{close_hour:02}:{close_minute:02}")
        return result

    # ✅ 다중 시간대 (ex: "09:30~11:30, 14:00~15:30")
    time_ranges = re.findall(
        r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
        reception_time,
    )

    for match in time_ranges:
        start_hour, start_minute, end_hour, end_minute = match
        start_hour = int(start_hour)
        end_hour = int(end_hour)
        start_minute = (
            int(start_minute) if start_minute and start_minute.isdigit() else 0
        )
        end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0

        # ✅ 오픈/마감 시간 분리하여 저장
        result["토"].append(f"{start_hour:02}:{start_minute:02}")
        result["토"].append(f"{end_hour:02}:{end_minute:02}")

    return result


# rcvWeek - 주간 접수시간
def normalize_reception_time(reception_time):
    """접수시간 데이터를 요일별 24시간 HH:MM~HH:MM 형식으로 변환"""

    if not reception_time or "정보 없음" in reception_time:
        return {"정보 없음": "정보 없음"}

    reception_time = reception_time.strip()

    # ✅ 요일 매핑
    weekday_map = {
        "월": "월",
        "화": "화",
        "수": "수",
        "목": "목",
        "금": "금",
        "토": "토",
        "일": "일",
    }

    reception_dict = {day: [] for day in weekday_map.values()}

    # ✅ "연중무휴" 처리 (모든 요일에 적용)
    if "연중무휴" in reception_time or "365일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            # ✅ 모든 요일에 적용
            for day in weekday_map.values():
                reception_dict[day] = [
                    f"{start_hour:02}:{start_minute:02}~{end_hour:02}:{end_minute:02}"
                ]
        reception_dict["연중무휴 여부"] = True
        return reception_dict

    # ✅ "평일" 포함된 경우 (월~금 적용)
    if "평일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            for day in ["월", "화", "수", "목", "금"]:
                reception_dict[day] = [
                    f"{start_hour:02}:{start_minute:02}",
                    f"{end_hour:02}:{end_minute:02}",
                ]

    # ✅ 요일별 접수 마감 시간 처리 (ex: "월,수:6시10분까지 , 화.금: 8시10분까지 접수, 목요일 휴진")
    weekday_closing = re.findall(
        r"([월화수목금토일,/]+)[:\s]*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*까지",
        reception_time,
    )

    if weekday_closing:
        for match in weekday_closing:
            days_raw, hour, minute = match
            days = re.split(r"[,/]", days_raw)  # '월,수' 또는 '목/토'를 개별적으로 분리
            hour = int(hour)
            minute = int(minute) if minute and minute.isdigit() else 0
            formatted_time = f"{hour:02}:{minute:02}"
            for d in days:
                if d.strip() in weekday_map:
                    reception_dict[weekday_map[d.strip()]].append(formatted_time)

    # ✅ 시간대 패턴 (ex: "09시30분~18시30분", "9시 - 19시(오후7시)")
    pattern = re.findall(
        r"([월화수목금토일,/]*)\s*(오전|오후)?\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(오전|오후)?\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
        reception_time,
    )

    for match in pattern:
        days_raw, ampm1, start_hour, start_minute, ampm2, end_hour, end_minute = match
        days = (
            re.split(r"[,/]", days_raw) if days_raw else ["월", "화", "수", "목", "금"]
        )
        start_hour = int(start_hour)
        end_hour = int(end_hour)
        start_minute = (
            int(start_minute) if start_minute and start_minute.isdigit() else 0
        )
        end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0

        # ✅ 오전/오후 변환 처리
        if ampm1 == "오후" and start_hour < 12:
            start_hour += 12
        if ampm2 == "오후" and end_hour < 12:
            end_hour += 12

        for d in days:
            d = d.strip()
            if d in weekday_map:
                reception_dict[weekday_map[d]].append(
                    f"{start_hour:02}:{start_minute:02}"
                )
                reception_dict[weekday_map[d]].append(f"{end_hour:02}:{end_minute:02}")

    # ✅ 목요일 휴진 처리
    if "목요일 휴진" in reception_time:
        reception_dict["목"].append("휴진")

    return {day: times for day, times in reception_dict.items() if times}  # 빈 값 제거


def preprocess_hospital_data(df: pd.DataFrame) -> List[Dict]:
    """병원 데이터를 JSON 형식으로 변환하는 함수"""
    processed_data = []

    for _, row in df.iterrows():
        hospital = {
            "점심시간": {
                "주중": normalize_lunch_time(row.get("lunchWeek", "정보 없음")),
                "토요일": normalize_lunch_time(row.get("lunchSat", "정보 없음")),
            },
            "휴무일": {
                "일요일": normalize_noTrmtSun(row.get("noTrmtSun", "정보 없음")),
                "공휴일": get_holiday_dates(),
            },
            "접수시간": {
                "주중": normalize_reception_time(row.get("rcvWeek", "정보 없음")),
                "토요일": normalize_rcvSat(row.get("rcvSat", "정보 없음")),
            },
            "진료시간": normalize_treatment_time(row),
        }
        processed_data.append(hospital)

    return processed_data


def save_to_json(data: List[Dict], filename: str = OUTPUT_FILE):
    """JSON 파일로 저장"""
    import json

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"📂 데이터가 {filename} 파일로 저장되었습니다.")


# ✅ 병원 데이터 불러오기 및 전처리 실행
if os.path.exists(DATA_PATH):
    df = pd.read_excel(DATA_PATH, engine="openpyxl")
    processed_data = preprocess_hospital_data(df)
    save_to_json(processed_data)
    print("✅ 데이터 전처리 완료 및 저장")
else:
    print(f"🚨 파일이 존재하지 않습니다: {DATA_PATH}")

In [ ]:
# 진료시간 계산
if hospital_data["진료시간"].get(day_of_week):
    start, end = hospital_data["진료시간"][day_of_week]
    start_time = datetime.strptime(start, "%H:%M")
    end_time = datetime.strptime(end, "%H:%M")

    # 현재 시간이 진료시간 내에 있는지 확인
    if start_time <= current_time <= end_time:
        print("✅ 현재 병원이 운영 중입니다!")
    else:
        print("❌ 현재 병원이 운영하지 않습니다.")
else:
    print("❌ 오늘은 휴진입니다.")

# 하나의 코드로 정리


In [ ]:
import openai
import pandas as pd
import os
import requests
import xml.etree.ElementTree as ET
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from xml.dom import minidom
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_core.prompts import ChatPromptTemplate
from holidayskr import is_holiday
from typing import Dict, List, Union
import json

# ✅ API 및 파일 설정
BASIS_URL = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"
DETAIL_URL = "http://apis.data.go.kr/B551182/MadmDtlInfoService2.7/getDtlInfo2.7"
API_KEY = "jB13be2Bs3Cy4foj2au1Y8dbOzJMrEhELtQg54is5jFE9EbDa1TND7ELLBe3L1YJ0rDDD5uPd87JAbGFcbicqA=="

# ✅ 파일 경로 설정
EXCEL_FILE = "hospitals_basic_data.xlsx"
FINAL_JSON_FILE = "processed_hospitals.json"
XML_FILE = "zero_coordinates_hospitals.xml"

# ✅ API 요청 설정
NUM_ROWS = 5000
TOTAL_COUNT = 78484
TOTAL_PAGES = (TOTAL_COUNT // NUM_ROWS) + 1
MAX_WORKERS = 10

# ✅ 사용자 위치
USER_LAT, USER_LON = 37.525782, 126.883021
RADIUS_M = 3000

# ✅ 전역 변수
zero_coordinates_hospitals = []

# ✅ 진료과목 코드
# ✅ 진료과목 코드 딕셔너리
medical_departments = {
    "일반의": "00",
    "내과": "01",
    "신경과": "02",
    "정신건강의학과": "03",
    "외과": "04",
    "정형외과": "05",
    "신경외과": "06",
    "심장혈관흉부외과": "07",
    "성형외과": "08",
    "마취통증의학과": "09",
    "산부인과": "10",
    "소아청소년과": "11",
    "안과": "12",
    "이비인후과": "13",
    "피부과": "14",
    "비뇨의학과": "15",
    "영상의학과": "16",
    "방사선종양학과": "17",
    "병리과": "18",
    "진단검사의학과": "19",
    "결핵과": "20",
    "재활의학과": "21",
    "핵의학과": "22",
    "가정의학과": "23",
    "응급의학과": "24",
    "직업환경의학과": "25",
    "예방의학과": "26",
    "기타1(치과)": "27",
    "기타4(한방)": "28",
    "기타2": "31",
    "기타2(2)": "40",
    "보건": "41",
    "기타3": "42",
    "보건기관치과": "43",
    "보건기관한방": "44",
    "치과": "49",
    "구강악안면외과": "50",
    "치과보철과": "51",
    "치과교정과": "52",
    "소아치과": "53",
    "치주과": "54",
    "치과보존과": "55",
    "구강내과": "56",
    "영상치의학과": "57",
    "구강병리과": "58",
    "예방치과": "59",
    "치과소계": "60",
    "통합치의학과": "61",
    "한방내과": "80",
    "한방부인과": "81",
    "한방소아과": "82",
    "한방안·이비인후·피부과": "83",
    "한방신경정신과": "84",
    "침구과": "85",
    "한방재활의학과": "86",
    "사상체질과": "87",
    "한방응급": "88",
    "한방응급": "89",
    "한방소계": "90",
}

# ✅ 시간 정보 컬럼
# ✅ 진료시간 정보 컬럼 리스트
COLUMNS = [
    "plcDir",
    "trmtSunEnd",
    "rcvSat",
    "trmtSunStart",
    "emyNgtTelNo1",
    "emyNgtTelNo2",
    "lunchWeek",
    "trmtWedStart",
    "trmtWedEnd",
    "trmtThuStart",
    "trmtMonStart",
    "trmtMonEnd",
    "trmtTueStart",
    "trmtTueEnd",
    "lunchSat",
    "rcvWeek",
    "trmtSatStart",
    "trmtSatEnd",
    "plcNm",
    "parkXpnsYn",
    "parkEtc",
    "noTrmtSun",
    "noTrmtHoli",
    "emyDayYn",
    "emyDayTelNo1",
    "emyDayTelNo2",
    "emyNgtYn",
    "trmtThuEnd",
    "trmtFriStart",
    "trmtFriEnd",
    "plcDist",
    "parkQty",
]


def get_medical_department(user_input: str) -> List[str]:
    """사용자 증상에 따른 진료과 추천"""
    if not user_input.strip():
        return ["🤖 챗봇: 증상을 입력해주세요."]

    # 프롬프트 생성
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """너는 아이케어 챗봇으로 한국의 병원 및 약국 검색 도우미입니다.
                사용자의 증상을 분석하여 적절한 진료과를 추천해 주세요.
                
                - 단, 아이가 아프더라도 무조건 소아과만 추천하지 말고 증상에 따라 추천해주세요.
                - 반드시 결과는 단답형으로 추천 진료과만 반환하고, 형식은 아래 예시처럼 유지하세요.
                
                예시:
                사용자 입력: "목이 아프고 열이 나요."
                챗봇 응답:
                ```
                이비인후과, 내과
                ```
                
                사용자 입력: "무릎이 너무 아파요."
                챗봇 응답:
                ```
                정형외과
                ```
                
                항상 한국어로 응답하고, 친절하고 간결하게 안내해주세요.
                """,
            ),
            ("human", "{input}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    formatted_prompt = prompt.format(input=user_input)
    response = llm.invoke(formatted_prompt)

    if response and response.content:
        content = response.content.strip().strip("```").strip()
        return content.split(", ")
    return ["🤖 챗봇: 죄송합니다. 다시 입력해주세요."]


def fetch_hospital_page(page_no, dgsbjtCd):
    """병렬로 특정 페이지의 병원 데이터를 가져오는 함수"""
    params = {
        "ServiceKey": API_KEY,
        "numOfRows": str(NUM_ROWS),
        "pageNo": str(page_no),
        "dgsbjtCd": dgsbjtCd,
        "xPos": str(USER_LON),
        "yPos": str(USER_LAT),
        "radius": str(RADIUS_M),
    }
    response = requests.get(BASIS_URL, params=params)

    if response.status_code != 200:
        print(f"🚨 API 호출 실패 (페이지 {page_no}): {response.status_code}")
        return []

    # XML → JSON 변환
    root = ET.fromstring(response.content)
    hospitals = []

    for item in root.findall(".//item"):
        xpos = float(item.findtext("XPos", "0"))
        ypos = float(item.findtext("YPos", "0"))

        hospital_data = {
            "ykiho": item.findtext("ykiho", "정보 없음"),  # 병원 고유 ID
            "yadmNm": item.findtext("yadmNm", "정보 없음"),  # 병원명
            "addr": item.findtext("addr", "정보 없음"),  # 주소
            "XPos": xpos,  # 위도
            "YPos": ypos,  # 경도
            "telno": item.findtext("telno", "정보 없음"),  # 전화번호
            "진료과목": [k for k, v in medical_departments.items() if v == dgsbjtCd][
                0
            ],  # 진료과목 한글로 추가
        }

        if xpos == 0 or ypos == 0:
            zero_coordinates_hospitals.append(item)  # ✅ 위경도 0인 병원 저장
        else:
            hospitals.append(hospital_data)  # ✅ 정상 병원만 저장

    return hospitals


# 진료시간 정보 추가
def fetch_hospital_details(ykiho):
    """특정 병원의 진료시간 및 추가 정보를 가져오기"""
    params = {"serviceKey": API_KEY, "ykiho": ykiho}
    response = requests.get(DETAIL_URL, params=params)

    if response.status_code != 200:
        return None  # API 오류 시 해당 병원 제외

    root = ET.fromstring(response.content)
    item = root.find(".//item")
    if item is None:
        return None  # 데이터가 없으면 제외

    # ✅ 데이터 추출
    hospital_info = {"ykiho": ykiho}
    for col in COLUMNS:
        hospital_info[col] = item.findtext(col, "정보 없음")  # 데이터 없으면 기본값

    return hospital_info


def fetch_all_hospitals(dgsbjtCd):
    """병렬 처리로 API에서 모든 병원 데이터를 가져오는 함수"""
    all_hospitals = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(
            executor.map(
                lambda page_no: fetch_hospital_page(page_no, dgsbjtCd),
                range(1, TOTAL_PAGES + 1),
            )
        )

    # ✅ 결과 병합
    for hospitals in results:
        all_hospitals.extend(hospitals)

    print(f"🚀 전체 병원 데이터 수집 완료! 총 {len(all_hospitals)}개 (위/경도 0 제외)")

    return all_hospitals


def process_hospitals(recommended_departments: List[str]) -> List[Dict]:
    """전체 병원 정보 수집 및 처리"""
    all_hospitals = []

    # 1. 기본 병원 정보 수집
    for department in recommended_departments:
        if department in medical_departments:
            dgsbjtCd = medical_departments[department]
            hospitals = fetch_all_hospitals(dgsbjtCd)

            # 2. 각 병원의 상세 정보 수집 및 처리
            for hospital in hospitals:
                details = fetch_hospital_details(hospital["ykiho"])
                if details:
                    processed_data = process_hospital_data(
                        hospital, details, department
                    )
                    all_hospitals.append(processed_data)

    return all_hospitals


# ==================================================
import re
from datetime import datetime, timedelta
from typing import Dict, List, Union
from holidayskr import is_holiday
import pandas as pd


def normalize_lunch_time(lunch_time: str) -> Union[None, List[str]]:
    """점심시간을 ['HH:MM', 'HH:MM'] 형식으로 변환."""
    if not lunch_time or "정보 없음" in lunch_time or "없음" in lunch_time:
        return None

    lunch_time = re.sub(r"[^0-9시분~:-]", "", lunch_time)
    time_match = re.findall(
        r"(\d{1,2})시?(\d{1,2})?분?\s*[~:-]\s*(\d{1,2})시?(\d{1,2})?분?", lunch_time
    )

    if time_match:
        start_hour, start_minute, end_hour, end_minute = time_match[0]
        start_hour = int(start_hour)
        start_minute = int(start_minute) if start_minute else 0
        end_hour = int(end_hour)
        end_minute = int(end_minute) if end_minute else 0
        return [f"{start_hour:02}:{start_minute:02}", f"{end_hour:02}:{end_minute:02}"]

    return None


def normalize_noTrmtSun(noTrmtSun: str) -> bool:
    """일요일 진료 여부를 반환 (휴진이면 True)"""
    if not noTrmtSun or "정보 없음" in noTrmtSun:
        return False
    return any(term in noTrmtSun for term in ["전부휴진", "휴진", "휴무"])


def get_holiday_dates(year: int = 2025) -> List[str]:
    """특정 연도의 모든 공휴일 날짜를 반환"""
    holidays = []
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    current_date = start_date
    while current_date <= end_date:
        if is_holiday(current_date.strftime("%Y-%m-%d")):
            holidays.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    return sorted(holidays)


def normalize_treatment_time(row: pd.Series) -> Dict[str, Union[List[str], None]]:
    """진료시간을 요일별 ['HH:MM', 'HH:MM'] 형식으로 변환"""
    # 요일 매핑
    day_mapping = {
        "Mon": "월",
        "Tue": "화",
        "Wed": "수",
        "Thu": "목",
        "Fri": "금",
        "Sat": "토",
        "Sun": "일",
    }

    def format_time(time_str: str) -> str:
        """시간 문자열을 HH:MM 형식으로 변환"""
        if isinstance(time_str, str) and len(time_str) == 4:
            return f"{time_str[:2]}:{time_str[2:]}"
        return "정보 없음"

    # 진료시간 정보 구성
    hospital_info = {}
    for eng_day, kor_day in day_mapping.items():
        start_time = format_time(row.get(f"trmt{eng_day}Start", "정보 없음"))
        end_time = format_time(row.get(f"trmt{eng_day}End", "정보 없음"))

        if start_time != "정보 없음" and end_time != "정보 없음":
            hospital_info[kor_day] = [start_time, end_time]
        else:
            hospital_info[kor_day] = None

    return hospital_info


# rcvWeek - 주간 접수시간
def normalize_reception_time(reception_time):
    """접수시간 데이터를 요일별 24시간 HH:MM~HH:MM 형식으로 변환"""

    if not reception_time or "정보 없음" in reception_time:
        return {"정보 없음": "정보 없음"}

    reception_time = reception_time.strip()

    # ✅ 요일 매핑
    weekday_map = {
        "월": "월",
        "화": "화",
        "수": "수",
        "목": "목",
        "금": "금",
        "토": "토",
        "일": "일",
    }

    reception_dict = {day: [] for day in weekday_map.values()}

    # ✅ "연중무휴" 처리 (모든 요일에 적용)
    if "연중무휴" in reception_time or "365일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            # ✅ 모든 요일에 적용
            for day in weekday_map.values():
                reception_dict[day] = [
                    f"{start_hour:02}:{start_minute:02}~{end_hour:02}:{end_minute:02}"
                ]
        reception_dict["연중무휴 여부"] = True
        return reception_dict

    # ✅ "평일" 포함된 경우 (월~금 적용)
    if "평일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            for day in ["월", "화", "수", "목", "금"]:
                reception_dict[day] = [
                    f"{start_hour:02}:{start_minute:02}",
                    f"{end_hour:02}:{end_minute:02}",
                ]

    # ✅ 요일별 접수 마감 시간 처리 (ex: "월,수:6시10분까지 , 화.금: 8시10분까지 접수, 목요일 휴진")
    weekday_closing = re.findall(
        r"([월화수목금토일,/]+)[:\s]*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*까지",
        reception_time,
    )

    if weekday_closing:
        for match in weekday_closing:
            days_raw, hour, minute = match
            days = re.split(r"[,/]", days_raw)  # '월,수' 또는 '목/토'를 개별적으로 분리
            hour = int(hour)
            minute = int(minute) if minute and minute.isdigit() else 0
            formatted_time = f"{hour:02}:{minute:02}"
            for d in days:
                if d.strip() in weekday_map:
                    reception_dict[weekday_map[d.strip()]].append(formatted_time)

    # ✅ 시간대 패턴 (ex: "09시30분~18시30분", "9시 - 19시(오후7시)")
    pattern = re.findall(
        r"([월화수목금토일,/]*)\s*(오전|오후)?\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(오전|오후)?\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
        reception_time,
    )

    for match in pattern:
        days_raw, ampm1, start_hour, start_minute, ampm2, end_hour, end_minute = match
        days = (
            re.split(r"[,/]", days_raw) if days_raw else ["월", "화", "수", "목", "금"]
        )
        start_hour = int(start_hour)
        end_hour = int(end_hour)
        start_minute = (
            int(start_minute) if start_minute and start_minute.isdigit() else 0
        )
        end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0

        # ✅ 오전/오후 변환 처리
        if ampm1 == "오후" and start_hour < 12:
            start_hour += 12
        if ampm2 == "오후" and end_hour < 12:
            end_hour += 12

        for d in days:
            d = d.strip()
            if d in weekday_map:
                reception_dict[weekday_map[d]].append(
                    f"{start_hour:02}:{start_minute:02}"
                )
                reception_dict[weekday_map[d]].append(f"{end_hour:02}:{end_minute:02}")

    # ✅ 목요일 휴진 처리
    if "목요일 휴진" in reception_time:
        reception_dict["목"].append("휴진")

    return {day: times for day, times in reception_dict.items() if times}  # 빈 값 제거


# rcvSat - 토요일 접수시간
def normalize_rcvSat(reception_time):
    """토요일 접수시간을 통일된 형식으로 변환 (HH:MM / HH:MM~HH:MM)"""

    if not reception_time or "정보 없음" in reception_time:
        return {"토": "정보 없음"}

    reception_time = reception_time.strip()
    result = {"토": []}

    # ✅ 연중무휴 처리 (토요일만 적용)
    if "연중무휴" in reception_time or "365일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            result["토"].append(f"{start_hour:02}:{start_minute:02}")
            result["토"].append(f"{end_hour:02}:{end_minute:02}")
        else:
            result["토"] = ["연중무휴"]
        result["연중무휴 여부"] = True
        return result

    # ✅ "HH:MM까지 접수" → 마감 시간만 처리
    closing_pattern = re.search(
        r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*까지\s*접수?", reception_time
    )
    if closing_pattern:
        close_hour, close_minute = closing_pattern.groups()
        close_hour = int(close_hour)
        close_minute = (
            int(close_minute) if close_minute and close_minute.isdigit() else 0
        )
        result["토"].append(f"{close_hour:02}:{close_minute:02}")
        return result

    # ✅ 다중 시간대 (ex: "09:30~11:30, 14:00~15:30")
    time_ranges = re.findall(
        r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
        reception_time,
    )

    for match in time_ranges:
        start_hour, start_minute, end_hour, end_minute = match
        start_hour = int(start_hour)
        end_hour = int(end_hour)
        start_minute = (
            int(start_minute) if start_minute and start_minute.isdigit() else 0
        )
        end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0

        # ✅ 오픈/마감 시간 분리하여 저장
        result["토"].append(f"{start_hour:02}:{start_minute:02}")
        result["토"].append(f"{end_hour:02}:{end_minute:02}")

    return result


# ==================================================


def process_hospital_data(hospital: Dict, details: Dict, department: str) -> Dict:
    """개별 병원 데이터 처리"""
    return {
        "병원명": hospital["yadmNm"],
        "주소": hospital["addr"],
        "전화번호": hospital["telno"],
        "위치": {"위도": hospital["XPos"], "경도": hospital["YPos"]},
        "진료과": department,
        "점심시간": {
            "주중": normalize_lunch_time(details.get("lunchWeek")),
            "토요일": normalize_lunch_time(details.get("lunchSat")),
        },
        "휴무일": {
            "일요일": normalize_noTrmtSun(details.get("noTrmtSun")),
            "공휴일": get_holiday_dates(),
        },
        "접수시간": {
            "주중": normalize_reception_time(details.get("rcvWeek")),
            "토요일": normalize_rcvSat(details.get("rcvSat")),
        },
        "진료시간": normalize_treatment_time(pd.Series(details)),
    }


def main():
    """메인 실행 함수"""
    # 1. 사용자 입력 받기
    user_input = input("어떤 증상으로 방문하시나요? ")
    recommended_departments = get_medical_department(user_input)
    print(f"추천 진료과: {recommended_departments}")

    # 2. 병원 정보 수집 및 처리
    processed_hospitals = process_hospitals(recommended_departments)

    # 3. 결과 저장
    with open(FINAL_JSON_FILE, "w", encoding="utf-8") as f:
        json.dump(processed_hospitals, f, ensure_ascii=False, indent=2)

    print(f"✅ 처리 완료! 총 {len(processed_hospitals)}개 병원 정보가 저장되었습니다.")


if __name__ == "__main__":
    main()

# 병렬처리

In [17]:
import openai
import pandas as pd
import os
import requests
import xml.etree.ElementTree as ET
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from xml.dom import minidom
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_core.prompts import ChatPromptTemplate
from holidayskr import is_holiday
from typing import Dict, List, Union
import json
import time

# ✅ API 및 파일 설정
BASIS_URL = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"
DETAIL_URL = "http://apis.data.go.kr/B551182/MadmDtlInfoService2.7/getDtlInfo2.7"
API_KEY = "jB13be2Bs3Cy4foj2au1Y8dbOzJMrEhELtQg54is5jFE9EbDa1TND7ELLBe3L1YJ0rDDD5uPd87JAbGFcbicqA=="

# ✅ 파일 경로 설정
EXCEL_FILE = "hospitals_basic_data.xlsx"
FINAL_JSON_FILE = "processed_hospitals.json"
XML_FILE = "zero_coordinates_hospitals.xml"

# ✅ API 요청 설정
NUM_ROWS = 5000
TOTAL_COUNT = 78484
TOTAL_PAGES = (TOTAL_COUNT // NUM_ROWS) + 1
MAX_WORKERS = 20
BATCH_SIZE = 100
RETRY_COUNT = 3
TIMEOUT = 10

# ✅ 사용자 위치
USER_LAT, USER_LON = 37.525782, 126.883021
RADIUS_M = 3000

# ✅ 전역 변수
zero_coordinates_hospitals = []

# ✅ 진료과목 코드
# ✅ 진료과목 코드 딕셔너리
medical_departments = {
    "일반의": "00",
    "내과": "01",
    "신경과": "02",
    "정신건강의학과": "03",
    "외과": "04",
    "정형외과": "05",
    "신경외과": "06",
    "심장혈관흉부외과": "07",
    "성형외과": "08",
    "마취통증의학과": "09",
    "산부인과": "10",
    "소아청소년과": "11",
    "안과": "12",
    "이비인후과": "13",
    "피부과": "14",
    "비뇨의학과": "15",
    "영상의학과": "16",
    "방사선종양학과": "17",
    "병리과": "18",
    "진단검사의학과": "19",
    "결핵과": "20",
    "재활의학과": "21",
    "핵의학과": "22",
    "가정의학과": "23",
    "응급의학과": "24",
    "직업환경의학과": "25",
    "예방의학과": "26",
    "기타1(치과)": "27",
    "기타4(한방)": "28",
    "기타2": "31",
    "기타2(2)": "40",
    "보건": "41",
    "기타3": "42",
    "보건기관치과": "43",
    "보건기관한방": "44",
    "치과": "49",
    "구강악안면외과": "50",
    "치과보철과": "51",
    "치과교정과": "52",
    "소아치과": "53",
    "치주과": "54",
    "치과보존과": "55",
    "구강내과": "56",
    "영상치의학과": "57",
    "구강병리과": "58",
    "예방치과": "59",
    "치과소계": "60",
    "통합치의학과": "61",
    "한방내과": "80",
    "한방부인과": "81",
    "한방소아과": "82",
    "한방안·이비인후·피부과": "83",
    "한방신경정신과": "84",
    "침구과": "85",
    "한방재활의학과": "86",
    "사상체질과": "87",
    "한방응급": "88",
    "한방응급": "89",
    "한방소계": "90",
}

# ✅ 시간 정보 컬럼
# ✅ 진료시간 정보 컬럼 리스트
COLUMNS = [
    "plcDir",
    "trmtSunEnd",
    "rcvSat",
    "trmtSunStart",
    "lunchWeek",
    "trmtWedStart",
    "trmtWedEnd",
    "trmtThuStart",
    "trmtMonStart",
    "trmtMonEnd",
    "trmtTueStart",
    "trmtTueEnd",
    "lunchSat",
    "rcvWeek",
    "trmtSatStart",
    "trmtSatEnd",
    "trmtThuEnd",
    "trmtFriStart",
    "trmtFriEnd",
    "noTrmtSun",
    "noTrmtHoli",
    
    "plcNm",
    "parkXpnsYn",
    "parkEtc",
    "emyDayYn",
    "emyDayTelNo1",
    "emyDayTelNo2",
    "emyNgtYn",
    "emyNgtTelNo1",
    "emyNgtTelNo2",
    "plcDist",
    "parkQty",
]


def get_medical_department(user_input: str) -> List[str]:
    """사용자 증상에 따른 진료과 추천"""
    if not user_input.strip():
        return ["🤖 챗봇: 증상을 입력해주세요."]

    # 프롬프트 생성
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """너는 아이케어 챗봇으로 한국의 병원 및 약국 검색 도우미입니다.
                사용자의 증상을 분석하여 적절한 진료과를 추천해 주세요.
                
                - 단, 아이가 아프더라도 무조건 소아과만 추천하지 말고 증상에 따라 추천해주세요.
                - 반드시 결과는 단답형으로 추천 진료과만 반환하고, 형식은 아래 예시처럼 유지하세요.
                
                예시:
                사용자 입력: "목이 아프고 열이 나요."
                챗봇 응답:
                ```
                이비인후과, 내과
                ```
                
                사용자 입력: "무릎이 너무 아파요."
                챗봇 응답:
                ```
                정형외과
                ```
                
                항상 한국어로 응답하고, 친절하고 간결하게 안내해주세요.
                """,
            ),
            ("human", "{input}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    formatted_prompt = prompt.format(input=user_input)
    response = llm.invoke(formatted_prompt)

    if response and response.content:
        content = response.content.strip().strip("```").strip()
        return content.split(", ")
    return ["🤖 챗봇: 죄송합니다. 다시 입력해주세요."]


def fetch_hospital_page(page_no, dgsbjtCd):
    """병렬로 특정 페이지의 병원 데이터를 가져오는 함수"""
    params = {
        "ServiceKey": API_KEY,
        "numOfRows": str(NUM_ROWS),
        "pageNo": str(page_no),
        "dgsbjtCd": dgsbjtCd,
        "xPos": str(USER_LON),
        "yPos": str(USER_LAT),
        "radius": str(RADIUS_M),
    }
    response = requests.get(BASIS_URL, params=params)

    if response.status_code != 200:
        print(f"🚨 API 호출 실패 (페이지 {page_no}): {response.status_code}")
        return []

    # XML → JSON 변환
    root = ET.fromstring(response.content)
    hospitals = []

    for item in root.findall(".//item"):
        xpos = float(item.findtext("XPos", "0"))
        ypos = float(item.findtext("YPos", "0"))

        hospital_data = {
            "ykiho": item.findtext("ykiho", "정보 없음"),  # 병원 고유 ID
            "yadmNm": item.findtext("yadmNm", "정보 없음"),  # 병원명
            "addr": item.findtext("addr", "정보 없음"),  # 주소
            "XPos": xpos,  # 위도
            "YPos": ypos,  # 경도
            "telno": item.findtext("telno", "정보 없음"),  # 전화번호
            "진료과목": [k for k, v in medical_departments.items() if v == dgsbjtCd][
                0
            ],  # 진료과목 한글로 추가
            "distance": item.findtext("distance", "정보 없음"),  # 사용자와의 거리
            
        }

        if xpos == 0 or ypos == 0:
            zero_coordinates_hospitals.append(item)  # ✅ 위경도 0인 병원 저장
        else:
            hospitals.append(hospital_data)  # ✅ 정상 병원만 저장

    return hospitals


# 진료시간 정보 추가
def fetch_hospital_details(ykiho):
    """특정 병원의 진료시간 및 추가 정보를 가져오기"""
    params = {"serviceKey": API_KEY, "ykiho": ykiho}
    response = requests.get(DETAIL_URL, params=params)

    if response.status_code != 200:
        return None  # API 오류 시 해당 병원 제외

    root = ET.fromstring(response.content)
    item = root.find(".//item")
    if item is None:
        return None  # 데이터가 없으면 제외

    # ✅ 데이터 추출
    hospital_info = {"ykiho": ykiho}
    for col in COLUMNS:
        hospital_info[col] = item.findtext(col, "정보 없음")  # 데이터 없으면 기본값

    return hospital_info


def fetch_all_hospitals(dgsbjtCd):
    """병렬 처리로 API에서 모든 병원 데이터를 가져오는 함수"""
    all_hospitals = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = list(
            executor.map(
                lambda page_no: fetch_hospital_page(page_no, dgsbjtCd),
                range(1, TOTAL_PAGES + 1),
            )
        )

    # ✅ 결과 병합
    for hospitals in results:
        all_hospitals.extend(hospitals)

    print(f"🚀 전체 병원 데이터 수집 완료! 총 {len(all_hospitals)}개 (위/경도 0 제외)")

    return all_hospitals


def process_hospitals(recommended_departments: List[str]) -> List[Dict]:
    """전체 병원 정보 수집 및 처리"""
    all_hospitals = []

    def process_hospital_batch(hospitals_batch):
        """병원 배치 처리"""
        processed_batch = []
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            # 병렬로 상세 정보 가져오기
            future_to_hospital = {
                executor.submit(fetch_hospital_details, hospital["ykiho"]): hospital
                for hospital in hospitals_batch
            }

            for future in as_completed(future_to_hospital):
                hospital = future_to_hospital[future]
                details = future.result()
                if details:
                    processed_data = process_hospital_data(
                        hospital, details, hospital["진료과목"]
                    )
                    processed_batch.append(processed_data)

        return processed_batch

    # 1. 모든 진료과의 병원 정보 수집
    for department in recommended_departments:
        if department in medical_departments:
            dgsbjtCd = medical_departments[department]
            hospitals = fetch_all_hospitals(dgsbjtCd)

            # 2. 배치 크기로 나누어 처리
            for i in range(0, len(hospitals), BATCH_SIZE):
                batch = hospitals[i : i + BATCH_SIZE]
                processed_batch = process_hospital_batch(batch)
                all_hospitals.extend(processed_batch)

                # 진행상황 표시
                print(
                    f"✅ 처리 완료: {len(all_hospitals)}개 병원 ({i + len(batch)}/{len(hospitals)})"
                )

    return all_hospitals


# ==================================================
import re
from datetime import datetime, timedelta
from typing import Dict, List, Union
from holidayskr import is_holiday
import pandas as pd


def normalize_lunch_time(lunch_time: str) -> Union[None, List[str]]:
    """점심시간을 ['HH:MM', 'HH:MM'] 형식으로 변환."""
    if not lunch_time or "정보 없음" in lunch_time or "없음" in lunch_time:
        return None

    lunch_time = re.sub(r"[^0-9시분~:-]", "", lunch_time)
    time_match = re.findall(
        r"(\d{1,2})시?(\d{1,2})?분?\s*[~:-]\s*(\d{1,2})시?(\d{1,2})?분?", lunch_time
    )

    if time_match:
        start_hour, start_minute, end_hour, end_minute = time_match[0]
        start_hour = int(start_hour)
        start_minute = int(start_minute) if start_minute else 0
        end_hour = int(end_hour)
        end_minute = int(end_minute) if end_minute else 0
        return [f"{start_hour:02}:{start_minute:02}", f"{end_hour:02}:{end_minute:02}"]

    return None


def normalize_noTrmtSun(noTrmtSun: str) -> bool:
    """일요일 진료 여부를 반환 (휴진이면 True)"""
    if not noTrmtSun or "정보 없음" in noTrmtSun:
        return False
    return any(term in noTrmtSun for term in ["전부휴진", "휴진", "휴무"])


def get_holiday_dates(year: int = 2025) -> List[str]:
    """특정 연도의 모든 공휴일 날짜를 반환"""
    holidays = []
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    current_date = start_date
    while current_date <= end_date:
        if is_holiday(current_date.strftime("%Y-%m-%d")):
            holidays.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    return sorted(holidays)


def normalize_treatment_time(row: pd.Series) -> Dict[str, Union[List[str], None]]:
    """진료시간을 요일별 ['HH:MM', 'HH:MM'] 형식으로 변환"""
    # 요일 매핑
    day_mapping = {
        "Mon": "월",
        "Tue": "화",
        "Wed": "수",
        "Thu": "목",
        "Fri": "금",
        "Sat": "토",
        "Sun": "일",
    }

    def format_time(time_str: str) -> str:
        """시간 문자열을 HH:MM 형식으로 변환"""
        if isinstance(time_str, str) and len(time_str) == 4:
            return f"{time_str[:2]}:{time_str[2:]}"
        return "정보 없음"

    # 진료시간 정보 구성
    hospital_info = {}
    for eng_day, kor_day in day_mapping.items():
        start_time = format_time(row.get(f"trmt{eng_day}Start", "정보 없음"))
        end_time = format_time(row.get(f"trmt{eng_day}End", "정보 없음"))

        if start_time != "정보 없음" and end_time != "정보 없음":
            hospital_info[kor_day] = [start_time, end_time]
        else:
            hospital_info[kor_day] = None

    return hospital_info


# rcvWeek - 주간 접수시간
def normalize_reception_time(reception_time):
    """접수시간 데이터를 요일별 24시간 HH:MM~HH:MM 형식으로 변환"""

    if not reception_time or "정보 없음" in reception_time:
        return {"정보 없음": "정보 없음"}

    reception_time = reception_time.strip()

    # ✅ 요일 매핑
    weekday_map = {
        "월": "월",
        "화": "화",
        "수": "수",
        "목": "목",
        "금": "금",
        "토": "토",
        "일": "일",
    }

    reception_dict = {day: [] for day in weekday_map.values()}

    # ✅ "연중무휴" 처리 (모든 요일에 적용)
    if "연중무휴" in reception_time or "365일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            # ✅ 모든 요일에 적용
            for day in weekday_map.values():
                reception_dict[day] = [
                    f"{start_hour:02}:{start_minute:02}~{end_hour:02}:{end_minute:02}"
                ]
        reception_dict["연중무휴 여부"] = True
        return reception_dict

    # ✅ "평일" 포함된 경우 (월~금 적용)
    if "평일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            for day in ["월", "화", "수", "목", "금"]:
                reception_dict[day] = [
                    f"{start_hour:02}:{start_minute:02}",
                    f"{end_hour:02}:{end_minute:02}",
                ]

    # ✅ 요일별 접수 마감 시간 처리 (ex: "월,수:6시10분까지 , 화.금: 8시10분까지 접수, 목요일 휴진")
    weekday_closing = re.findall(
        r"([월화수목금토일,/]+)[:\s]*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*까지",
        reception_time,
    )

    if weekday_closing:
        for match in weekday_closing:
            days_raw, hour, minute = match
            days = re.split(r"[,/]", days_raw)  # '월,수' 또는 '목/토'를 개별적으로 분리
            hour = int(hour)
            minute = int(minute) if minute and minute.isdigit() else 0
            formatted_time = f"{hour:02}:{minute:02}"
            for d in days:
                if d.strip() in weekday_map:
                    reception_dict[weekday_map[d.strip()]].append(formatted_time)

    # ✅ 시간대 패턴 (ex: "09시30분~18시30분", "9시 - 19시(오후7시)")
    pattern = re.findall(
        r"([월화수목금토일,/]*)\s*(오전|오후)?\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(오전|오후)?\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
        reception_time,
    )

    for match in pattern:
        days_raw, ampm1, start_hour, start_minute, ampm2, end_hour, end_minute = match
        days = (
            re.split(r"[,/]", days_raw) if days_raw else ["월", "화", "수", "목", "금"]
        )
        start_hour = int(start_hour)
        end_hour = int(end_hour)
        start_minute = (
            int(start_minute) if start_minute and start_minute.isdigit() else 0
        )
        end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0

        # ✅ 오전/오후 변환 처리
        if ampm1 == "오후" and start_hour < 12:
            start_hour += 12
        if ampm2 == "오후" and end_hour < 12:
            end_hour += 12

        for d in days:
            d = d.strip()
            if d in weekday_map:
                reception_dict[weekday_map[d]].append(
                    f"{start_hour:02}:{start_minute:02}"
                )
                reception_dict[weekday_map[d]].append(f"{end_hour:02}:{end_minute:02}")

    # ✅ 목요일 휴진 처리
    if "목요일 휴진" in reception_time:
        reception_dict["목"].append("휴진")

    return {day: times for day, times in reception_dict.items() if times}  # 빈 값 제거


# rcvSat - 토요일 접수시간
def normalize_rcvSat(reception_time):
    """토요일 접수시간을 통일된 형식으로 변환 (HH:MM / HH:MM~HH:MM)"""

    if not reception_time or "정보 없음" in reception_time:
        return {"토": "정보 없음"}

    reception_time = reception_time.strip()
    result = {"토": []}

    # ✅ 연중무휴 처리 (토요일만 적용)
    if "연중무휴" in reception_time or "365일" in reception_time:
        pattern = re.search(
            r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
            reception_time,
        )
        if pattern:
            start_hour, start_minute, end_hour, end_minute = pattern.groups()
            start_hour = int(start_hour)
            end_hour = int(end_hour)
            start_minute = (
                int(start_minute) if start_minute and start_minute.isdigit() else 0
            )
            end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0
            result["토"].append(f"{start_hour:02}:{start_minute:02}")
            result["토"].append(f"{end_hour:02}:{end_minute:02}")
        else:
            result["토"] = ["연중무휴"]
        result["연중무휴 여부"] = True
        return result

    # ✅ "HH:MM까지 접수" → 마감 시간만 처리
    closing_pattern = re.search(
        r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*까지\s*접수?", reception_time
    )
    if closing_pattern:
        close_hour, close_minute = closing_pattern.groups()
        close_hour = int(close_hour)
        close_minute = (
            int(close_minute) if close_minute and close_minute.isdigit() else 0
        )
        result["토"].append(f"{close_hour:02}:{close_minute:02}")
        return result

    # ✅ 다중 시간대 (ex: "09:30~11:30, 14:00~15:30")
    time_ranges = re.findall(
        r"(\d{1,2})시?[:\s]*(\d{1,2})?[분]*\s*[~\-]\s*(\d{1,2})시?[:\s]*(\d{1,2})?[분]*",
        reception_time,
    )

    for match in time_ranges:
        start_hour, start_minute, end_hour, end_minute = match
        start_hour = int(start_hour)
        end_hour = int(end_hour)
        start_minute = (
            int(start_minute) if start_minute and start_minute.isdigit() else 0
        )
        end_minute = int(end_minute) if end_minute and end_minute.isdigit() else 0

        # ✅ 오픈/마감 시간 분리하여 저장
        result["토"].append(f"{start_hour:02}:{start_minute:02}")
        result["토"].append(f"{end_hour:02}:{end_minute:02}")

    return result


# ==================================================


def process_hospital_data(hospital: Dict, details: Dict, department: str) -> Dict:
    """개별 병원 데이터 처리"""
    return {
        "병원명": hospital["yadmNm"],
        "주소": hospital["addr"],
        "전화번호": hospital["telno"],
        "위치": {"위도": hospital["XPos"], "경도": hospital["YPos"]},
        "위치": hospital["distance"],
        "진료과": department,
        "점심시간": {
            "주중": normalize_lunch_time(details.get("lunchWeek")),
            "토요일": normalize_lunch_time(details.get("lunchSat")),
        },
        "휴무일": {
            "일요일": normalize_noTrmtSun(details.get("noTrmtSun")),
            "공휴일": get_holiday_dates(),
        },
        "접수시간": {
            "주중": normalize_reception_time(details.get("rcvWeek")),
            "토요일": normalize_rcvSat(details.get("rcvSat")),
        },
        "진료시간": normalize_treatment_time(pd.Series(details)),
    }


def main():
    """메인 실행 함수"""
    # 1. 사용자 입력 받기
    user_input = input("어떤 증상으로 방문하시나요? ")
    departments = get_medical_department(user_input)
    print(f"추천 진료과: {departments}")

    # 2. 병원 정보 수집
    processed_hospitals = process_hospitals(departments)

    # 3. 결과 저장
    with open(FINAL_JSON_FILE, "w", encoding="utf-8") as f:
        json.dump(processed_hospitals, f, ensure_ascii=False, indent=2)

    print(f"✅ 처리 완료! 총 {len(processed_hospitals)}개 병원 정보가 저장되었습니다.")
    print(f"📂 저장 위치: {os.path.abspath(FINAL_JSON_FILE)}")

    # 4. 저장된 결과 확인
    try:
        with open(FINAL_JSON_FILE, "r", encoding="utf-8") as f:
            saved_data = json.load(f)
            print(f"\n✅ 저장된 데이터 확인 (첫 번째 병원):")
            print(json.dumps(saved_data[0], ensure_ascii=False, indent=2))
    except Exception as e:
        print(f"🚨 파일 읽기 실패: {str(e)}")


if __name__ == "__main__":
    main()
    
    
import json
from datetime import datetime
from math import radians, sin, cos, sqrt, atan2

# JSON 파일 읽기
with open('processed_hospitals.json', 'r', encoding='utf-8') as f:
    hospitals = json.load(f)

# 현재 시간과 요일
current_time = datetime.now()
day_of_week = current_time.strftime("%a")  # 요일을 영문 약자로 가져옴 (Mon, Tue, Wed, ...)

# 요일 매핑
day_mapping = {
    "Mon": "월",
    "Tue": "화",
    "Wed": "수",
    "Thu": "목",
    "Fri": "금",
    "Sat": "토",
    "Sun": "일"
}

# 현재 요일에 해당하는 한글 요일
day_of_week_kor = day_mapping[day_of_week]

# 현재 문이 열린 병원 리스트
open_hospitals = []

# 사용자 위치 (위도, 경도)
user_lat = 37.525782
user_lon = 126.883021

def parse_time(time_str):
    """시간 문자열을 datetime 객체로 변환 (24:00을 00:00으로 변환)"""
    if time_str == "24:00":
        time_str = "00:00"
    return datetime.strptime(time_str, "%H:%M")

def calculate_distance(lat1, lon1, lat2, lon2):
    """두 지점 간의 거리를 계산 (단위: km)"""
    R = 6371.0  # 지구 반지름 (단위: km)

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

for hospital in hospitals:
    is_open = False
    status = "영업 준비 중"
    operating_hours = None

    # 진료시간 확인
    if hospital["진료시간"].get(day_of_week_kor):
        start, end = hospital["진료시간"][day_of_week_kor]
        start_time = parse_time(start)
        end_time = parse_time(end)
        operating_hours = f"{start}~{end}"

        # 현재 시간이 진료시간 내에 있는지 확인
        if start_time.time() <= current_time.time() <= end_time.time():
            is_open = True
            status = "영업 중"

    # 진료시간이 없으면 접수시간 확인
    if not is_open and hospital["접수시간"].get(day_of_week_kor):
        for reception_time in hospital["접수시간"][day_of_week_kor]:
            start, end = reception_time.split("~")
            start_time = parse_time(start)
            end_time = parse_time(end)
            operating_hours = f"{start}~{end}"

            # 현재 시간이 접수시간 내에 있는지 확인
            if start_time.time() <= current_time.time() <= end_time.time():
                is_open = True
                status = "영업 중"

    # 공휴일 확인
    if "공휴일" in hospital["휴무일"] and current_time.strftime("%Y-%m-%d") in hospital["휴무일"]["공휴일"]:
        is_open = False
        status = "휴무일"

    # 휴무일 확인
    if "일요일" in hospital["휴무일"] and hospital["휴무일"]["일요일"] and day_of_week_kor == "일":
        is_open = False
        status = "휴무일"

    if is_open:
        distance = calculate_distance(user_lat, user_lon, hospital["위치"]["위도"], hospital["위치"]["경도"])
        open_hospitals.append({
            "병원명": hospital["병원명"],
            "주소": hospital["주소"],
            "전화번호": hospital["전화번호"],
            "진료과목": hospital["진료과"],
            "진료시간": operating_hours,
            "상태": status,
            "거리": distance
        })

# 거리순으로 정렬
open_hospitals.sort(key=lambda x: x["거리"])

# 결과 출력
print(f"현재 문이 열린 병원 수: {len(open_hospitals)}")
for hospital in open_hospitals:
    print(f"병원명: {hospital['병원명']}, 주소: {hospital['주소']}, 전화번호: {hospital['전화번호']}, 진료과목: {hospital['진료과목']}, 진료시간: {hospital['진료시간']}, 상태: {hospital['상태']}, 거리: {hospital['거리']:.2f} km")

추천 진료과: ['이비인후과', '내과', '소화기내과']
🚀 전체 병원 데이터 수집 완료! 총 158개 (위/경도 0 제외)
✅ 처리 완료: 33개 병원 (100/158)
✅ 처리 완료: 49개 병원 (158/158)
🚀 전체 병원 데이터 수집 완료! 총 297개 (위/경도 0 제외)
✅ 처리 완료: 92개 병원 (100/297)
✅ 처리 완료: 119개 병원 (200/297)
✅ 처리 완료: 142개 병원 (297/297)
✅ 처리 완료! 총 142개 병원 정보가 저장되었습니다.
📂 저장 위치: c:\Users\RMARKET\workspace\project\backend\chat\test\processed_hospitals.json

✅ 저장된 데이터 확인 (첫 번째 병원):
{
  "병원명": "한림대학교 한강성심병원",
  "주소": "서울특별시 영등포구 버드나루로7길 12, (영등포동7가, 한강성심병원)",
  "전화번호": "02-2639-5114",
  "위치": "2387.748018030125409991878895923100124",
  "진료과": "이비인후과",
  "점심시간": {
    "주중": [
      "12:00",
      "00:00"
    ],
    "토요일": null
  },
  "휴무일": {
    "일요일": false,
    "공휴일": [
      "2025-01-01",
      "2025-01-27",
      "2025-01-28",
      "2025-01-29",
      "2025-01-30",
      "2025-03-01",
      "2025-03-03",
      "2025-05-01",
      "2025-05-05",
      "2025-05-06",
      "2025-06-06",
      "2025-08-15",
      "2025-10-03",
      "2025-10-05",
      "2025-10-06",
      "2025-10-07",


In [16]:
import json
from datetime import datetime
from math import radians, sin, cos, sqrt, atan2

# JSON 파일 읽기
with open('processed_hospitals.json', 'r', encoding='utf-8') as f:
    hospitals = json.load(f)

# 현재 시간과 요일
current_time = datetime.now()
day_of_week = current_time.strftime("%a")  # 요일을 영문 약자로 가져옴 (Mon, Tue, Wed, ...)

# 요일 매핑
day_mapping = {
    "Mon": "월",
    "Tue": "화",
    "Wed": "수",
    "Thu": "목",
    "Fri": "금",
    "Sat": "토",
    "Sun": "일"
}

# 현재 요일에 해당하는 한글 요일
day_of_week_kor = day_mapping[day_of_week]

# 현재 문이 열린 병원 리스트
open_hospitals = []

# 사용자 위치 (위도, 경도)
user_lat = 37.525782
user_lon = 126.883021

def parse_time(time_str):
    """시간 문자열을 datetime 객체로 변환 (24:00을 00:00으로 변환)"""
    if time_str == "24:00":
        time_str = "00:00"
    return datetime.strptime(time_str, "%H:%M")

def calculate_distance(lat1, lon1, lat2, lon2):
    """두 지점 간의 거리를 계산 (단위: km)"""
    R = 6371.0  # 지구 반지름 (단위: km)

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

for hospital in hospitals:
    is_open = False
    status = "영업 준비 중"
    operating_hours = None

    # 진료시간 확인
    if hospital["진료시간"].get(day_of_week_kor):
        start, end = hospital["진료시간"][day_of_week_kor]
        start_time = parse_time(start)
        end_time = parse_time(end)
        operating_hours = f"{start}~{end}"

        # 현재 시간이 진료시간 내에 있는지 확인
        if start_time.time() <= current_time.time() <= end_time.time():
            is_open = True
            status = "영업 중"

    # 진료시간이 없으면 접수시간 확인
    if not is_open and hospital["접수시간"].get(day_of_week_kor):
        for reception_time in hospital["접수시간"][day_of_week_kor]:
            start, end = reception_time.split("~")
            start_time = parse_time(start)
            end_time = parse_time(end)
            operating_hours = f"{start}~{end}"

            # 현재 시간이 접수시간 내에 있는지 확인
            if start_time.time() <= current_time.time() <= end_time.time():
                is_open = True
                status = "영업 중"

    # 공휴일 확인
    if "공휴일" in hospital["휴무일"] and current_time.strftime("%Y-%m-%d") in hospital["휴무일"]["공휴일"]:
        is_open = False
        status = "휴무일"

    # 휴무일 확인
    if "일요일" in hospital["휴무일"] and hospital["휴무일"]["일요일"] and day_of_week_kor == "일":
        is_open = False
        status = "휴무일"

    if is_open:
        distance = calculate_distance(user_lat, user_lon, hospital["위치"]["위도"], hospital["위치"]["경도"])
        open_hospitals.append({
            "병원명": hospital["병원명"],
            "주소": hospital["주소"],
            "전화번호": hospital["전화번호"],
            "진료과목": hospital["진료과"],
            "진료시간": operating_hours,
            "상태": status,
            "거리": distance
        })

# 거리순으로 정렬
open_hospitals.sort(key=lambda x: x["거리"])

# 결과 출력
print(f"현재 문이 열린 병원 수: {len(open_hospitals)}")
for hospital in open_hospitals:
    print(f"병원명: {hospital['병원명']}, 주소: {hospital['주소']}, 전화번호: {hospital['전화번호']}, 진료과목: {hospital['진료과목']}, 진료시간: {hospital['진료시간']}, 상태: {hospital['상태']}, 거리: {hospital['거리']:.2f} km")

현재 문이 열린 병원 수: 118
병원명: 서울드림정형외과의원, 주소: 서울특별시 구로구 고척로 239, 힘찬테크 4층 (고척동), 전화번호: 02-2066-3060, 진료과목: 내과, 진료시간: 09:00~18:30, 상태: 영업 중, 거리: 6802.01 km
병원명: 서울스마트요양병원, 주소: 서울특별시 양천구 중앙로 181, 복합메디컬타운 3~8층 (신정동), 전화번호: 02-2601-7111, 진료과목: 내과, 진료시간: 08:30~17:30, 상태: 영업 중, 거리: 6802.20 km
병원명: 별소아청소년과의원, 주소: 서울특별시 양천구 신정로 312, 뉴프라자 404호 (신정동), 전화번호: 02-2694-1527, 진료과목: 이비인후과, 진료시간: 09:00~19:00, 상태: 영업 중, 거리: 6802.22 km
병원명: 별소아청소년과의원, 주소: 서울특별시 양천구 신정로 312, 뉴프라자 404호 (신정동), 전화번호: 02-2694-1527, 진료과목: 내과, 진료시간: 09:00~19:00, 상태: 영업 중, 거리: 6802.22 km
병원명: 연세드림이비인후과의원, 주소: 서울특별시 양천구 목동남로4길 2, 2층 205호 (신정동, 세양청마루2차 주상복합), 전화번호: 02-2688-7313, 진료과목: 이비인후과, 진료시간: 09:00~18:30, 상태: 영업 중, 거리: 6802.34 km
병원명: 연세드림이비인후과의원, 주소: 서울특별시 양천구 목동남로4길 2, 2층 205호 (신정동, 세양청마루2차 주상복합), 전화번호: 02-2688-7313, 진료과목: 내과, 진료시간: 09:00~18:30, 상태: 영업 중, 거리: 6802.34 km
병원명: 달팽이이비인후과의원, 주소: 서울특별시 양천구 중앙로 262, 메디바이오플렉스 502,503호 (신정동), 전화번호: 02-2062-4298, 진료과목: 이비인후과, 진료시간: 09:00~19:00, 상태: 영업 중, 거리: 6802.52 km
병원명: 고려정형외과의원, 주소: 서울